# Road Test

In [1]:
import emat
emat.__version__

'0.1.1'

In [2]:
import ema_workbench
import plotly_nb
import os, numpy, pandas, functools
from xmle import Show

In [3]:
logger = emat.util.loggers.log_to_stderr(20, True)

## Defining the Exploratory Scope

In [4]:
road_test_scope_file = emat.package_file('model','tests','road_test.yaml')

In [5]:
road_scope = emat.Scope(road_test_scope_file)
road_scope

<emat.Scope with 2 constants, 7 uncertainties, 4 levers, 7 measures>

A short summary of the scope can be reviewed using the `info` method.

In [6]:
road_scope.info()

name: EMAT Road Test
desc: prototype run
constants:
  free_flow_time = 60
  initial_capacity = 100
uncertainties:
  alpha = 0.1 to 0.2
  beta = 3.5 to 5.5
  input_flow = 80 to 150
  value_of_time = 0.001 to 0.08
  unit_cost_expansion = 95 to 145
  interest_rate = 0.025 to 0.04
  yield_curve = -0.0025 to 0.02
levers:
  expand_capacity = 0 to 100
  amortization_period = 15 to 50
  debt_type = categorical
  interest_rate_lock = boolean
measures:
  no_build_travel_time
  build_travel_time
  time_savings
  value_of_time_savings
  net_benefits
  cost_of_capacity_expansion
  present_cost_expansion


Alternatively, more detailed information about each part of the scope can be
accessed in four list attributes:

In [7]:
road_scope.get_constants()

[Parameter('free_flow_time', dtype=real, ptype='constant'),
 Parameter('initial_capacity', dtype=real, ptype='constant')]

In [8]:
road_scope.get_uncertainties()

[Parameter('alpha', dtype=real, ptype='uncertainty'),
 Parameter('beta', dtype=real, ptype='uncertainty'),
 Parameter('input_flow', dtype=int, ptype='uncertainty'),
 Parameter('value_of_time', dtype=real, ptype='uncertainty'),
 Parameter('unit_cost_expansion', dtype=real, ptype='uncertainty'),
 Parameter('interest_rate', dtype=real, ptype='uncertainty'),
 Parameter('yield_curve', dtype=real, ptype='uncertainty')]

In [9]:
road_scope.get_levers()

[Parameter('expand_capacity', dtype=real, ptype='lever'),
 Parameter('amortization_period', dtype=int, ptype='lever'),
 Parameter('debt_type', dtype=cat, ptype='lever'),
 Parameter('interest_rate_lock', dtype=bool, ptype='lever')]

In [10]:
road_scope.get_measures()

[Measure('no_build_travel_time'),
 Measure('build_travel_time'),
 Measure('time_savings'),
 Measure('value_of_time_savings'),
 Measure('net_benefits'),
 Measure('cost_of_capacity_expansion'),
 Measure('present_cost_expansion')]

## Using a Database

The exploratory modeling process will typically generate many different sets of outputs,
for different explored modeling scopes, or for different applications.  It is convenient
to organize these outputs in a database structure, so they are stored consistently and 
readily available for subsequent analysis.

The `SQLiteDB` object will create a database to store results.  When instantiated with
no arguments, the database is initialized in-memory, which will not store anything to
disk (which is convenient for this example, but in practice you will generally want to
store data to disk so that it can persist after this Python session ends).

In [11]:
emat_db = emat.SQLiteDB()

[00:04.99] MainProcess/INFO: running script scope.sql
[00:05.00] MainProcess/INFO: running script exp_design.sql
[00:05.00] MainProcess/INFO: running script meta_model.sql


An EMAT Scope can be stored in the database, to provide needed information about what the 
various inputs and outputs represent.

In [12]:
road_scope.store_scope(emat_db)

Trying to store another scope with the same name (or the same scope) raises a KeyError.

In [13]:
try:
    road_scope.store_scope(emat_db)
except KeyError as err:
    print(err)

'scope named "EMAT Road Test" already exists'


We can review the names of scopes already stored in the database using the `read_scope_names` method.

In [14]:
emat_db.read_scope_names()

['EMAT Road Test']

## Experimental Design

Actually running the model can be done by the user on an *ad hoc* basis (i.e., manually defining every 
combination of inputs that will be evaluated) but the real power of EMAT comes from runnning the model
using algorithm-created experimental designs.

An important experimental design used in exploratory modeling is the Latin Hypercube.  This design selects
a random set of experiments across multiple input dimensions, to ensure "good" coverage of the 
multi-dimensional modeling space.

The `design_latin_hypercube` function creates such a design based on a `Scope`, and optionally
stores the design of experiments in a database.

In [15]:
from emat.experiment.experimental_design import design_experiments

In [16]:
design = design_experiments(road_scope, db=emat_db, n_samples_per_factor=10, sampler='lhs')
design.head()

,alpha,amortization_period,beta,debt_type,expand_capacity,input_flow,interest_rate,interest_rate_lock,unit_cost_expansion,value_of_time,yield_curve,free_flow_time,initial_capacity
experiment,,,,,,,,,,,,,
1,0.184682,38,5.237143,Rev Bond,18.224793,115,0.031645,0,118.213466,0.022518,0.015659,60,100
2,0.166133,36,4.121963,Paygo,87.525790,129,0.037612,1,141.322696,0.052306,0.007307,60,100
3,0.198937,44,4.719838,GO Bond,45.698048,105,0.028445,0,97.783320,0.011923,-0.001545,60,100
4,0.158758,42,4.915816,GO Bond,51.297546,113,0.036234,1,127.224901,0.077527,0.004342,60,100
5,0.157671,42,3.845952,Paygo,22.824149,133,0.039257,0,107.820482,0.027270,0.001558,60,100


In [17]:
large_design = design_experiments(road_scope, db=emat_db, n_samples=5000, sampler='lhs', design_name='lhs_large')
large_design.head()

,alpha,amortization_period,beta,debt_type,expand_capacity,input_flow,interest_rate,interest_rate_lock,unit_cost_expansion,value_of_time,yield_curve,free_flow_time,initial_capacity
experiment,,,,,,,,,,,,,
111,0.154130,21,5.061648,Rev Bond,75.542217,112,0.029885,1,124.452736,0.020584,0.001425,60,100
112,0.148731,29,4.088663,Rev Bond,91.184595,145,0.028659,0,131.688623,0.017927,0.007850,60,100
113,0.124027,34,3.956884,Paygo,60.436585,80,0.038101,0,95.462532,0.014444,0.011101,60,100
114,0.129724,41,4.969628,Paygo,74.271040,139,0.029665,0,98.206495,0.013711,0.010072,60,100
115,0.185723,22,4.485432,Paygo,61.084166,95,0.039195,0,140.792308,0.068903,0.019277,60,100


We can review what experimental designs have already been stored in the database using the 
`read_design_names` method of the `Database` object.

In [18]:
emat_db.read_design_names('EMAT Road Test')

['lhs', 'lhs_large']

## Core Model in Python

### Model Definition

In the simplest approach for EMAT, a model can be defined as a basic Python function, which accepts all
inputs (exogenous uncertainties, policy levers, and externally defined constants) as named keyword
arguments, and returns a dictionary where the dictionary keys are names of performace measures, and 
the mapped values are the computed values for those performance measures.  The `Road_Capacity_Investment`
function provided in EMAT is an example of such a function.  This made-up example considers the 
investment in capacity expansion for a single roadway link.  The inputs to this function are described
above in the Scope, including uncertain parameters in the volume-delay function,
traffic volumes, value of travel time savings, unit construction costs, and interest rates, and policy levers including the 
amount of capacity expansion and amortization period.

In [19]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment

In [20]:
from emat.model.core_python import PythonCoreModel
m = PythonCoreModel(Road_Capacity_Investment, scope=road_scope, db=emat_db)

### Model Execution

In [21]:
from ema_workbench import SequentialEvaluator

In [22]:
with SequentialEvaluator(m) as eval_seq:
    lhs_results = m.run_experiments(design_name='lhs', evaluator=eval_seq)
lhs_results.head()

[00:06.62] MainProcess/INFO: performing 110 scenarios/policies * 1 model(s) = 110 experiments
[00:06.63] MainProcess/INFO: performing experiments sequentially
[00:06.64] MainProcess/INFO: 11 cases completed
[00:06.64] MainProcess/INFO: 22 cases completed
[00:06.65] MainProcess/INFO: 33 cases completed
[00:06.65] MainProcess/INFO: 44 cases completed
[00:06.65] MainProcess/INFO: 55 cases completed
[00:06.66] MainProcess/INFO: 66 cases completed
[00:06.66] MainProcess/INFO: 77 cases completed
[00:06.66] MainProcess/INFO: 88 cases completed
[00:06.67] MainProcess/INFO: 99 cases completed
[00:06.67] MainProcess/INFO: 110 cases completed
[00:06.67] MainProcess/INFO: experiments finished


,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,
1,0.184682,5.237143,115,0.022518,118.213466,0.031645,0.015659,18.224793,38,Rev Bond,False,83.038716,69.586789,13.451927,34.835362,-79.515515,114.350877,2154.415985
2,0.166133,4.121963,129,0.052306,141.322696,0.037612,0.007307,87.525790,36,Paygo,True,88.474313,62.132583,26.341730,177.741192,-205.321480,383.062672,12369.380535
3,0.198937,4.719838,105,0.011923,97.783320,0.028445,-0.001545,45.698048,44,GO Bond,False,75.027180,62.543328,12.483852,15.629037,-151.944318,167.573355,4468.506839
4,0.158758,4.915816,113,0.077527,127.224901,0.036234,0.004342,51.297546,42,GO Bond,True,77.370428,62.268768,15.101660,132.298475,-167.624871,299.923347,6526.325171
5,0.157671,3.845952,133,0.027270,107.820482,0.039257,0.001558,22.824149,42,Paygo,False,88.328990,72.848428,15.480561,56.146908,-3.972940,60.119848,2460.910705


In [23]:
# with SequentialEvaluator(m) as eval_seq:
#     lhs_large_results = m.run_experiments(design_name='lhs_large', evaluator=eval_seq)
# lhs_large_results.head()

Once a particular design has been run once, the results can be recovered from the database without re-running the model itself.

In [24]:
reload_results = m.read_experiments(design_name='lhs')
reload_results.head()

,free_flow_time,initial_capacity,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,,,,,,,,,,,,
1,60.0,100.0,0.184682,5.237143,115,0.022518,118.213466,0.031645,0.015659,18.224793,38,Rev Bond,False,83.0387,69.5868,13.4519,34.8354,-79.5155,114.351,2154.42
2,60.0,100.0,0.166133,4.121963,129,0.052306,141.322696,0.037612,0.007307,87.525790,36,Paygo,True,88.4743,62.1326,26.3417,177.741,-205.321,383.063,12369.4
3,60.0,100.0,0.198937,4.719838,105,0.011923,97.783320,0.028445,-0.001545,45.698048,44,GO Bond,False,75.0272,62.5433,12.4839,15.629,-151.944,167.573,4468.51
4,60.0,100.0,0.158758,4.915816,113,0.077527,127.224901,0.036234,0.004342,51.297546,42,GO Bond,True,77.3704,62.2688,15.1017,132.298,-167.625,299.923,6526.33
5,60.0,100.0,0.157671,3.845952,133,0.027270,107.820482,0.039257,0.001558,22.824149,42,Paygo,False,88.329,72.8484,15.4806,56.1469,-3.97294,60.1198,2460.91


It is also possible to load only the parameters, or only the performance meausures.

In [25]:
lhs_params = m.read_experiment_parameters(design_name='lhs')
lhs_params.head()

,free_flow_time,initial_capacity,alpha,beta,input_flow,value_of_time,unit_cost_expansion,interest_rate,yield_curve,expand_capacity,amortization_period,debt_type,interest_rate_lock
experiment,,,,,,,,,,,,,
1,60.0,100.0,0.184682,5.237143,115,0.022518,118.213466,0.031645,0.015659,18.224793,38,Rev Bond,False
2,60.0,100.0,0.166133,4.121963,129,0.052306,141.322696,0.037612,0.007307,87.525790,36,Paygo,True
3,60.0,100.0,0.198937,4.719838,105,0.011923,97.783320,0.028445,-0.001545,45.698048,44,GO Bond,False
4,60.0,100.0,0.158758,4.915816,113,0.077527,127.224901,0.036234,0.004342,51.297546,42,GO Bond,True
5,60.0,100.0,0.157671,3.845952,133,0.027270,107.820482,0.039257,0.001558,22.824149,42,Paygo,False


In [26]:
lhs_outcomes = m.read_experiment_measures(design_name='lhs')
lhs_outcomes.head()

,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,
1,83.038716,69.586789,13.451927,34.835362,-79.515515,114.350877,2154.415985
2,88.474313,62.132583,26.341730,177.741192,-205.321480,383.062672,12369.380535
3,75.027180,62.543328,12.483852,15.629037,-151.944318,167.573355,4468.506839
4,77.370428,62.268768,15.101660,132.298475,-167.624871,299.923347,6526.325171
5,88.328990,72.848428,15.480561,56.146908,-3.972940,60.119848,2460.910705


### CART

Classification and Regression Trees (CART) can also be used for scenario discovery. 
They partition the explored space (i.e., the scope) into a number of sections, with each partition
being added in such a way as to maximize the difference between observations on each 
side of the newly added partition divider, subject to some constraints.

In [27]:
# from ema_workbench.analysis import cart

# cart_alg = cart.CART(
#     m.read_experiment_parameters(design_name='lhs_large'), 
#     m.read_experiment_measures(design_name='lhs_large')['net_benefits']>0,
# )
# cart_alg.build_tree()

In [28]:
# Show(cart_alg.show_tree(format='svg')) 

In [29]:
# cart_alg.boxes_to_dataframe(include_stats=True)

# Constraints

In [30]:
from emat import Constraint

The common use case for constraints in robust optimation is imposing requirements
on solution outcomes. For example, we may want to limit our robust search only
to solutions where the expected present cost of the capacity expansion is less
than some particular value (in our example here, 4000).  

In [31]:
constraint_1 = Constraint(
    "Maximum build_travel_time", 
    outcome_names="build_travel_time",
    function=Constraint.must_be_less_than(70),
)

Our second constraint is based exclusively on an input: the capacity expansion
must be at least 10.  We could also achieve this kind of constraint by changing
the exploratory scope, but we don't necessarily want to change the scope to 
conduct a single robust optimization analysis with a constraint on a policy lever.

In [32]:
constraint_2 = Constraint(
    "Minimum Capacity Expansion", 
    parameter_names="expand_capacity",
    function=Constraint.must_be_greater_than(20),
)

It is also possible to impose constraints based on a combination of inputs and outputs.
For example, suppose that the total funds available for pay-as-you-go financing are
only 1500.  We may thus want to restrict the robust search to only solutions that
are almost certainly within the available funds at 99% confidence (a model output) but only 
if the Paygo financing option is used (a model input).  This kind of constraint can
be created by giving both `parameter_names` and `outcomes_names`, and writing a constraint
function that takes two arguments.

In [33]:
constraint_3 = Constraint(
    "Maximum Paygo Present Cost", 
    parameter_names='debt_type',
    outcome_names='present_cost_expansion',
    function=lambda i,j: max(0, j-4000) if i=='Paygo' else 0,
)

In [34]:
constraints=[
            constraint_1,
            constraint_2,
            constraint_3,
        ]

In [35]:
from emat.util.constraints import batch_contraint_check

In [36]:
batch_contraint_check(constraints, lhs_params, lhs_outcomes, False).head()

,Maximum build_travel_time,Minimum Capacity Expansion,Maximum Paygo Present Cost
experiment,,,
1,True,False,True
2,True,True,False
3,True,True,True
4,True,True,True
5,False,True,True


In [37]:
from emat.scope.box import Bounds, Box, Boxes, find_all_boxes_with_parent

In [38]:
db = emat_db

In [39]:
try:
    s = Box(scope=road_scope)
except TypeError:
    print("correct error")
else:
    raise RuntimeError

correct error


In [40]:
s = Box(
    name="Speedy", 
    scope=road_scope,
    upper_bounds={'build_travel_time':70},
    relevant=['net_benefits', 'time_savings'],
)

In [41]:
s2 = Box(
    name="Notable", 
    scope=road_scope, 
    parent="Speedy",
    lower_bounds={'expand_capacity': 20},
    relevant=road_scope.get_lever_names(),
)

In [42]:
s3 = Box(
    name="No Tax Dollars",
    scope=road_scope, 
    parent="Notable",
    allowed={
        'debt_type': {'Paygo', 'Rev Bond'},
        'interest_rate_lock': {False},
    }
)

In [43]:
u = Boxes(s,s2,s3, scope=road_scope)

In [44]:
u.fancy_names()

['Scope: EMAT Road Test', '▶ Speedy', '▷ ▶ Notable', '▷ ▷ ▶ No Tax Dollars']

In [45]:
u.get_chain("No Tax Dollars")

ChainedBox: No Tax Dollars
   ●    build_travel_time <= 70
   ●      expand_capacity >= 20
   ●            debt_type: {'Rev Bond', 'Paygo'}
   ●   interest_rate_lock: {False}
   ◌   interest_rate_lock
   ◌      expand_capacity
   ◌  amortization_period
   ◌            debt_type

In [46]:
print(u.get_chain("No Tax Dollars").chain_repr())

Box: Speedy
   ●  build_travel_time <= 70
Box: Notable
   ●      expand_capacity >= 20
   ◌   interest_rate_lock
   ◌      expand_capacity
   ◌  amortization_period
   ◌            debt_type
Box: No Tax Dollars
   ●           debt_type: {'Rev Bond', 'Paygo'}
   ●  interest_rate_lock: {False}


In [47]:
db.write_boxes(u)

In [48]:
uu = db.read_boxes()

In [49]:
uu.fancy_names()

['Boxes Universe', '▶ Speedy', '▷ ▶ Notable', '▷ ▷ ▶ No Tax Dollars']

In [56]:
uu.get_chain("No Tax Dollars")

ChainedBox: No Tax Dollars
   ●    build_travel_time <= 70
   ●            debt_type: {'Rev Bond', 'Paygo'}
   ●   interest_rate_lock: {0}
   ◌      expand_capacity
   ◌  amortization_period
   ◌            debt_type

In [57]:
db.read_scope('EMAT Road Test')

<emat.Scope with 2 constants, 7 uncertainties, 4 levers, 7 measures>

In [58]:
from emat.interactive import Explorer

In [59]:
explore = Explorer('db')
explore

Explorer(children=(VBox(children=(HBox(children=(Output(layout=Layout(flex='1 1 0%', width='auto')), Button(de…

In [54]:
from emat.interactive import prototype_logging
prototype_logging.handler.out

Output(layout=Layout(border='1px solid red'))

In [55]:
prototype_logging.logger.setLevel(10)

In [61]:
True in {0}

False